Lets download the open source graspnet implementatino and try to achieve some results...

Outcomes:

1. Verify pretrained model works
2. Training model from scratch
3. Gym Wrapper around the graspnet dataset (this one is really important!) * I am not sure if this is needed... well just for eval
4. Training model from scratch with RL loss function



---

SKIPPING THIS FOR NOW AS THE GRASPNET DATASET SEEMS TO HAVE SERIOUS PROBLEMS! GRASPS ARE IN COLLISION!

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# https://github.com/TylerYep/torchinfo
from torchinfo import summary 

np.random.seed(1)

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
# Load Data
train_data = datasets.MNIST(root="/home/bam/bam_ws/src/bam_brain/bam_gym/data_downloads", train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST(root='/home/bam/bam_ws/src/bam_brain/bam_gym/data_downloads', train=False, download=True, transform=transforms.ToTensor())
print("Training set size:", len(train_data))
print("Test set size:", len(test_data))
print(type(train_data[0][0]), type(train_data[0][1])) 

In [ ]:
# Viz Data
fig, axes = plt.subplots(1, 3, figsize=(10, 3))  

for ax, i in zip(axes, np.random.randint(0, 60000, 3)):
    image, label = train_data[i]
    ax.imshow(image.squeeze(0).numpy(), cmap='gray')
    ax.set_title(f'Label: {label}')
    ax.axis('off')

plt.tight_layout()
plt.show()

plt.title('Histogram of Pixel Values')
plt.hist(train_data[0][0].numpy().flatten(), bins=10)
plt.show()

In [ ]:
# Cross Val Split

from torch.utils.data import random_split
val_size = 0.2

val_subset, train_subset = random_split(train_data, [val_size, 1-val_size])
print(type(val_subset))
print(type(train_subset))
print("Number of samples in val data:", len(val_subset))
print("Number of samples in train data:", len(train_subset))


In [ ]:
train_dataloader = DataLoader(
    train_subset,
    batch_size=256,
    shuffle=True,
    num_workers=4,                # 4 workers load batches in parallel
    pin_memory=(device=="cuda"),  # Pin host memory for fast async H2D copies
    persistent_workers=True,      # Keep workers alive across epochs
    prefetch_factor=2             # Each worker preloads 2 batches
)

val_loader = DataLoader(
    val_subset,
    batch_size=256,
    shuffle=True,
    num_workers=4,                # 4 workers load batches in parallel
    pin_memory=(device=="cuda"),  # Pin host memory for fast async H2D copies
    persistent_workers=True,      # Keep workers alive across epochs
    prefetch_factor=2             # Each worker preloads 2 batches
)

test_dataloader = DataLoader(
    test_data,
    batch_size=256,
    shuffle=True,
    num_workers=4,                # 4 workers load batches in parallel
    pin_memory=(device=="cuda"),  # Pin host memory for fast async H2D copies
    persistent_workers=True,      # Keep workers alive across epochs
    prefetch_factor=2             # Each worker preloads 2 batches
)

for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

In [ ]:
"""
Implements the forward propagation for the multiclass classification model:
ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MaxPool2D -> FLATTEN -> LazyLinear -> Softmax

Arguments:
None

Returns:
model -- PyTorch Sequential container
"""

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_stack = nn.Sequential(
            nn.ZeroPad2d(2),
            nn.Conv2d(1, 16, 5, 1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.LazyLinear(out_features=10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        logits = self.conv_stack(x)
        return logits

model = CNN().to(device)
# print(model)
summary(model, input_size=(2, 1, 28, 28))

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer)

In [ ]:
def train(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer, device: str):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X) 
        train_loss: torch.Tensor = loss_fn(pred, y) 

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = train_loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def validate(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, device: str):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [ ]:
for epoch in range(10):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, device)
    validate(val_loader, model, loss_fn, device)


In [ ]:
save_path = "/home/bam/bam_ws/src/bam_brain/bam_gym/saved_models/ex3_mnist_model.pth"
torch.save(model.state_dict(), save_path)

In [ ]:
model = CNN().to(device)
model.load_state_dict(torch.load(save_path))

In [ ]:

def preprocess_state(state: np.ndarray):
    return (
        torch.tensor(state, dtype=torch.float32)  # convert to float tensor
        .unsqueeze(0)  # add batch dim
        .unsqueeze(0)  # add channel dim
        .to(device)    # move to same device as model
    )

class CNNPolicy():
    def __init__(self, model: nn.Module, preprocess_state: Callable, epsilon=0.2):

        self.model = model
        self.epsilon = epsilon
        self.iter = 0
        self.preprocess_state = preprocess_state


    def __call__(self, state: np.ndarray):
        self.iter += 1

        state = self.preprocess_state(state)


        action_values = self.model(state)

        if np.random.random() <= self.epsilon:
            return np.random.randint(action_values.shape[0])
        else:
            return torch.argmax(action_values).item()


policy = CNNPolicy(model, preprocess_state, epsilon=0.2)

policy(np.random.uniform(0, 1, (28, 28)))


In [ ]:
import gymnasium as gym
import bam_gym # import to that __init__.py is run to register envs
from bam_gym import print_reset, print_step

In [ ]:

env = gym.make("bam_local/MNISTEnv", render_mode="none", dataset=train_data)
print("Action Space:", env.action_space)
print("Observation Space:", env.observation_space)
state, info = env.reset()
print("Reset:", state.shape)


In [ ]:
def reset_env(env, terminated, truncated, info) -> bool:
    if terminated or truncated:
        if env.metadata['autoreset_mode'] != gym.vector.AutoresetMode.SAME_STEP:
            return True

    return False

In [ ]:
state, info = env.reset()
print_reset(state, info)

for i in range(10):
    action = env.action_space.sample()
    state, reward, terminated, truncated, info = env.step(action)
    print_step(action, state, reward, terminated, truncated, info, i)

    if reset_env(env, terminated, truncated, info):
        state, info = env.reset()

env.close()


In [ ]:
# env.close()

In [ ]:
reward_history = []
epsilon_history = []

In [ ]:
state, info = env.reset()
policy.epsilon = 0.0 # Manually tune epsilon 

for i in range(10000):
    action = policy(state)
    new_state, reward, terminated, truncated, info = env.step(action)
    print_step(action, state, reward, terminated, truncated, info, i)
    reward_history.append(reward)
    epsilon_history.append(policy.epsilon)
    state = new_state

    if reset_env(env, terminated, truncated, info):
        state, info = env.reset()



In [ ]:
plt.plot(reward_history, label="Reward")
plt.plot(epsilon_history, label="Epsilon", color='red')
# Compute moving average
window_size = 50
if len(reward_history) >= window_size:
    moving_avg = np.convolve(reward_history, np.ones(window_size)/window_size, mode='valid')
    plt.plot(range(window_size-1, len(reward_history)), moving_avg, label=f"Moving Avg ({window_size})", color='orange')

plt.xlabel("Episode")
plt.ylabel("Reward")
plt.title("Reward History and Moving Average")
plt.legend()
plt.show()